In [1]:
import numpy as np
import collections

# Definition of the states, accions y parameters
alpha = 0.3 # 30%

down = -5
up = 5
# State space
X = [i for i in range (down, up+1)]

# Control space 
U = [1, 0]

# Latest time
T = 3

In [2]:
def AVAR(q, w, alpha):
    # we recive w = w_{t+1}

    # this is something like {w(1): 20%, w(): 60%}
    eval = [(w[max(down, min(k, up))], q[k])  for k in q.keys()]
    s = sorted(eval, reverse=True)
    res = 0
    i = 0
    a = alpha
    while(alpha > 0):
        if alpha >= s[i][1]:
            res += s[i][0]*s[i][1]
            alpha -= s[i][1]
        else:
            res += s[i][0]*alpha
            alpha = 0
        i += 1
    return res/a


def precal_Q(X):
    Q = dict()
    for x in X:
        Q[x] = dict()
        Q[x][0] = dict()
        Q[x][0][0] = {x-1: 0.2, x-1: 0.2, x: 0.2, x+1: 0.2, x+2: 0.2}
        Q[x][0][1] = {x+1: 0.4, x+2: 0.2, x+3: 0.4}
        
        Q[x][1] = dict()
        Q[x][1][0] = {x-1: 0.6, x: 0.2, x+1: 0.2}
        Q[x][1][1] = {x-1: 0.2, x: 0.4, x+1: 0.4}
    return Q


Q_ = precal_Q(X)

def Q(s, x, d):
    # s: 
    # x: state
    # d: decision or policy
    return Q_[x][s][d]

In [6]:
class Node:
    def __init__(self, t, states, s = None, father = None):
        self.t = t # Time
        self.s = s # Action of the father
        self.father = father
        self.w = dict() # cost
        self.childs = dict()
        self.policy = dict() # One for each of the posible states 
        self.terminal = True # If it is a terminal node
        self.states = states # Posible states


    def cost(self, s, x, a):
        # s = Action of the father
        # x = state
        # a = action of this node
        t = self.t
        if s == 0 and a == 0:
            return 0
        if s == 0 and a == 1:
            return 1
        if s == 1: # Cot of terminal nodes in this case
            return np.exp(-x) - t 
        

    def new_child(self, node, s):
        self.childs[s] = node # Add the new child specifying which type is it
        node.father = self
        self.terminal = False 

    
    def calc_w(self):
        if self.terminal:
            for x in X: 
                self.w[x] = np.exp(-x) - self.t # Cost of terminal nodes in this case
                # This w is w_{t, s}
        else:
            self.calc_policy()
        
        return self.w

    def w_(self):
        if self.w.__len__() == 0:
            self.calc_w()
        return self.w


    def calc_policy(self):
        s = self.s
        for x in X: 
            self.policy[x] = min(U, key=lambda u: self.cost(s, x, u) + sum([AVAR(Q(s, x, u), child.w_(), alpha) for child in self.childs.values()]))
            u = self.policy[x]
            self.w[x] = self.cost(s, x, u) + sum([AVAR(Q(s, x, u), child.w_(), alpha) for child in self.childs.values()])


    def print_tree(self, level = 0):
        print(" " * 4 * level + f'x_{self.t, self.s}')
        for child in self.childs.values():
            child.print_tree(level = level+1)
            

In [60]:
# Create tree
#father = root

# ad_list is symilar to: [(0, 1), (0, 2), (1, 3), ...]
# With the relation that if we have (0,1), then 0 is father of 1
# To use this you have to have a node list and a list of which type they are
def create_tree(nodes, ad_list, types):
    root = nodes[0]
    for i, ad in enumerate(ad_list):
        nodes[ad[0]].new_child(nodes[ad[1]], types[i])
    return root


#adyacent list
ad_list = []

# Types of nodes
types = [0]

#The root
root = Node(0, states = X, s = 0)

# List of nodes
nodes = [root]

# We create the adyacent list for this case
for t in range(0, T-1):
    for u in U:
        new = Node(t+1, states = X, s = u) 
        nodes.append(new)
    ad_list.append((2*t, 2*t+1))
    ad_list.append((2*t, 2*t+2))
    types.append(1)
    types.append(0)

# Add the last node
new = Node(T, states = X, s = 1) 
nodes.append(new)
l = len(ad_list)
ad_list.append((l, l+1))
types.append(1)


print(ad_list)
print(types)

create_tree(nodes, ad_list, types)

'''for t in range(1, T):
    for u in U:
        new = Node(t, states = X, s = u, father=father) 
        father.new_child(new, u) 
    father = father.childs[0] # Be sure to define the order correctly the tree
    t = t + 1

# Add the last one
new = Node(T+1, states = X, s=1, father=father) '''


[(0, 1), (0, 2), (2, 3), (2, 4), (4, 5)]
[0, 1, 0, 1, 0, 1]


'for t in range(1, T):\n    for u in U:\n        new = Node(t, states = X, s = u, father=father) \n        father.new_child(new, u) \n    father = father.childs[0] # Be sure to define the order correctly the tree\n    t = t + 1\n\n# Add the last one\nnew = Node(T+1, states = X, s=1, father=father) '

In [61]:
root.print_tree()

x_(0, 0)
    x_(1, 1)
    x_(1, 0)
        x_(2, 1)
        x_(2, 0)
            x_(3, 1)


In [9]:
print(root.calc_w())
print(root.policy)

{-5: 91.76922387951957, -4: 31.86364912104897, -3: 9.825619755848741, -2: 1.718281828459045, -1: -1.2642411176571153, 0: -2.361449992355332, 1: -3.2839352730437232, 2: -3.7365745084047433, 3: -4.4348150181640635, 4: -4.792080064723704, 5: -4.923510530402154}
{-5: 1, -4: 1, -3: 1, -2: 1, -1: 1, 0: 1, 1: 1, 2: 1, 3: 0, 4: 0, 5: 0}


In [ ]:
# Define costs
def cost(t, s, x, a):
    if s == 0 and a == 0:
        return 0
    if s == 0 and a == 1:
        return 1
    if s == 1: # Cost of terminal nodes in this case
        return np.exp(-x) - t 


# To pre calculate all terminal nodes and save them
# w_{t,1}
def precalc_terminals_w(X, T):
    w = dict()
    for t in range(T+1):
        s = t
        s_ = dict()
        for u in U:
            s_[u] = dict()
        
        # This is for this case only
        for x in X: 
            s_[1][x] = np.exp(-x) - t # Cost of terminal nodes in this case
        w[s] = s_

    return w

def precal_Q(X):
    Q = dict()
    for x in X:
        Q[x] = dict()
        Q[x][0] = dict()
        Q[x][0][0] = {x-1: 0.2, x-1: 0.2, x: 0.2, x+1: 0.2, x+2: 0.2}
        Q[x][0][1] = {x+1: 0.4, x+2: 0.2, x+3: 0.4}
        
        Q[x][1] = dict()
        Q[x][1][0] = {x-1: 0.6, x: 0.2, x+1: 0.2}
        Q[x][1][1] = {x-1: 0.2, x: 0.4, x+1: 0.4}
    return Q

# In this case we have w_{t, 1} where 1 is the control space chosen
w = precalc_terminals_w(X, T)
# w[t][1]
w[0] = 0


Q_ = precal_Q(X)

In [ ]:
def Q(s, x, d): #Q^{t,s}(x, d) # In this case, q does not depend on t
    return Q_[x][s][d]

In [ ]:
def AVAR(q, w, alpha):
    # we recive w = w_{t+1}

    # this is something like {w(1): 20%, w(): 60%}
    eval = [(w[max(down, min(k, up))], q[k])  for k in q.keys()]
    s = sorted(eval, reverse=True)
    res = 0
    i = 0
    a = alpha
    while(alpha > 0):
        if alpha >= s[i][1]:
            res += s[i][0]*s[i][1]
            alpha -= s[i][1]
        else:
            res += s[i][0]*alpha
            alpha = 0
        i += 1
    return res/a

In [ ]:
policy = {}
t = 2
s = 0
while t > 0:
    for x in X:
        if t == 2:
            policy[x] = min(U, key=lambda u: cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha))
            u = policy[x]
            w[t][s][x] = cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha)
        else:
            policy[x] = min(U, key=lambda u: cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha) + AVAR(Q(s, x, u), w[t+1][0], alpha))
            u = policy[x]
            w[t][s][x] = cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha) + AVAR(Q(s, x, u), w[t+1][0], alpha)

        print(f'w{t, s, x} =, {w[t][s][x]}, policy: {u} ')
    t = t-1